In [1]:
import ray
#ray.shutdown()
#ray.init()

In [3]:
from files import BCTrade_utils
from azureml.core import Workspace,Dataset

ws = Workspace.from_config()
name ='Bitstamp_2'
dataset = Dataset.get_by_name(ws, name=name)
env = BCTrade_utils.BCTrade(dataset.to_pandas_dataframe(),singlerun=False,spreadfactor=0.0)
experiment_name = BCTrade_utils.registerBCTrade(name)
from ray.tune.registry import register_env
def env_creator(env_config):
    return BCTrade_utils.BCTrade(dataset.to_pandas_dataframe())
register_env("BCTrade-Bitstamp_2", env_creator)

In [4]:
#datastore = ws.get_default_datastore()
#checkpoint_ds = Dataset.File.from_files(path=[(datastore,'checkpoints_BCTrade-Btc_with_indicators_1641664748_e3baadf3_head/checkpoint_370')])
#checkpoint = checkpoint_ds.as_named_input('test')
#checkpoint.as_mount('xxx')
#checkpoint = '/tmp/checkpoint_370/checkpoint-370'
#Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_100
checkpoint = '/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/'
checkpoint+='logs/PPO/PPO_BCTrade-Bitstamp_1_c78d8870_2022-02-16_19-51-06i65hzgpu/checkpoint_595/checkpoint-595'
#checkpoint = 'logs/PPO/PPO_BCTrade-Btc_with_indicators_5a87271e_2022-01-09_18-24-16nmvj7oep/checkpoint_110/checkpoint-110'

In [5]:
import ray
import ray.rllib.agents.ppo as ppo

#ray.init()
#config = {
  #"callbacks": {
  #  "on_train_result": "<function on_train_result at 0x7fca737a8e18>"
  #},
#  "env": "BCTrade-Btc_with_indicators",
#  "num_gpus": 0,
#  "num_workers": 1
#}

config = ppo.DEFAULT_CONFIG.copy()
config["num_gpus"] = 0
config["num_workers"] = 1
#config["eager"] = False
config["env"] = "BCTrade-Bitstamp_2"
new_trainer = ppo.PPOTrainer(config=config,env="BCTrade-Bitstamp_2")
new_trainer


2022-02-17 16:48:14,731	INFO services.py:1340 -- View the Ray dashboard at http://127.0.0.1:8265
(RolloutWorker pid=30831) /anaconda/envs/azureml_py36/lib/python3.6/site-packages/gym/spaces/box.py:74: UserWarning: WARN: Box bound precision lowered by casting to float32
(RolloutWorker pid=30831)   "Box bound precision lowered by casting to {}".format(self.dtype)
(RolloutWorker pid=30831) WARNING:tensorflow:From /anaconda/envs/azureml_py36/lib/python3.6/site-packages/tensorflow_core/python/ops/resource_variable_ops.py:1635: calling BaseResourceVariable.__init__ (from tensorflow.python.ops.resource_variable_ops) with constraint is deprecated and will be removed in a future version.
(RolloutWorker pid=30831) Instructions for updating:
(RolloutWorker pid=30831) If using Keras pass *_constraint arguments to layers.
(RolloutWorker pid=30831) 2022-02-17 16:48:38,642	WARNING deprecation.py:46 -- DeprecationWarning: `SampleBatch['is_training']` has been deprecated. Use `SampleBatch.is_training` 

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


PPO

In [6]:
import pickle
with open(checkpoint, "rb") as f:
        extra_data = pickle.load(f)

extra_data['worker']
weights=pickle.loads(extra_data['worker'])
weights['state']['default_policy']
new_trainer.get_policy().set_weights(weights['state']['default_policy'])
#new_trainer.get_policy().set_state(weights['state'])
#new_trainer.restore(checkpoint)
#ray.tune.run(ppo.PPOTrainer, config=config, restore='/home/azureuser/cloudfiles/code/Users/rnikolaus/stuff/logs/PPO/PPO_BCTrade-Btc_with_indicators_95fc9bdc_2022-01-10_21-38-0176w5xe80/checkpoint_822')

In [7]:
#obs = env.initialize()
obs = env.reset()
rewardsum=0
c=0
while True:
	action = new_trainer.compute_action(obs, explore=False)
	if obs[0]==1 and action ==2:
		env.render()
	obs, reward, done, _ = env.step(action)
	rewardsum+=reward
	c+=1
	if done:
		break
env.render()
print(rewardsum,rewardsum/c,c)


2022-02-17 16:49:11,055	WARNING deprecation.py:46 -- DeprecationWarning: `compute_action` has been deprecated. Use `compute_single_action` instead. This will raise an error in the future!


1000.0000000000001
998.721795290334
997.6693263544098
998.8152716793542
996.7826000230145
997.3296957946966
997.541239791781
997.6446058431119
996.5794321606382
997.3719072159952
996.8672282863974
996.7867414832075
998.171892691724
995.9943141687777
995.8235298524656
996.6538123942237
996.6717459574891
996.2185657723942
997.1598584695038
996.7495312392306
996.5597362390646
997.9974927584208
997.2612610465757
997.2852267349963
996.925919244941
996.9674004941511
998.1763482717854
999.6329944066888
1000.0523494209202
1000.1071881787727
999.8458480340963
1000.3008399854853
1000.638624083448
997.7161732250997
996.5016754915946
997.7904655712397
998.5116263746484
997.9422930816716
998.1009411811636
997.0646195169772
997.2549353114583
997.3131819987675
996.1071048001799
996.0468350283091
996.8005781413954
997.0823939949832
997.3541229297055
997.726782287339
997.4752748810014
997.4083632060392
996.9683095819126
997.9929359034979
998.0051835057368
997.4237367704709
996.9504235159648
999.4430157